In [57]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
#settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/isabell/Documents/KTH/KTH femman/Scalable Machine learning/id2223-lab1


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [58]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [65]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()- datetime.timedelta(days=5)

location_str

2025-11-13 09:29:34,601 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-13 09:29:34,633 INFO: Initializing external client
2025-11-13 09:29:34,634 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-13 09:29:35,944 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1272015


'{"country": "sweden", "city": "solna", "street": "anders-lundstroms-gata", "aqicn_url": "https://api.waqi.info/feed/A61420", "latitude": "59.36004", "longitude": "18.00086"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [66]:
# Retrieve feature groups

#For original
air_quality_fg = fs.get_feature_group( 
    name='air_quality',
    version=1,
)

#For grade C
air_quality_fg_with_lagged = fs.get_feature_group( 
    name='air_quality_including_lagged',
    version=1,
)

weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [67]:
import requests
import pandas as pd

#For original
aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

#For C-level
aq__with_lagg_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq__with_lagg_today_df


,pm25,country,city,street,date,url
0,4.0,sweden,solna,anders-lundstroms-gata,2025-11-08,https://api.waqi.info/feed/A61420


In [68]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 👀 Assignment for level C: Adding lagged air quality data </span>

Adding the pm25 value for previous day, two days ago and three days ago as features.

In [83]:
#Reading yesterday's features and loads into pandas
# Build a query that selects only pm25
commits = air_quality_fg_with_lagged.commit_details()
print(commits)
query = air_quality_fg_with_lagged.select(["pm25"])

# Add a filter on the date column
yesterday = (today - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
query = query.filter(air_quality_fg_with_lagged.date == yesterday)

# Materialize the query into a DataFrame
df_yesterday = query.read()

#Now use yesterdays values and lagged values to update today's lagged values
aq__with_lagg_today_df["lagged_aq_1_day"] = yesterday_df["pm25"]
aq__with_lagg_today_df["lagged_aq_2_days"] = two_days_ago_df["pm25"]
aq__with_lagg_today_df["lagged_aq_3_days"] = three_days_ago_df["pm25"]

aq__with_lagg_today_df

{}
2025-11-13 10:19:52,174 ERROR: [Errno 2] Opening HDFS file '/apps/hive/warehouse/id2223lab1isagro_featurestore.db/air_quality_including_lagged_1/.hoodie/hoodie.properties' failed. Detail: [errno 2] No such file or directory. Detail: Python exception: Traceback (most recent call last):
  File "/usr/src/app/src/server.py", line 142, in wrapper
    result = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/src/app/src/server.py", line 166, in wrapper
    result = func(instance, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/src/app/src/server.py", line 196, in do_get
    return self._read_query(context, path, command)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/src/app/src/server.py", line 123, in wrapper
    return func(instance, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/src/app/src/server.py", line 131, in wrapper
    result = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^
  

FeatureStoreException: Could not read data using Hopsworks Query Service.

## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [ ]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 59.25°N 18.0°E
Elevation 13.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-13,9.60,0.2,15.542662,256.607483,solna
1,2025-11-14,3.05,0.0,11.275530,286.699310,solna
2,2025-11-15,1.50,0.0,11.720751,259.380402,solna
3,2025-11-16,3.25,0.0,11.440979,245.854462,solna
4,2025-11-17,0.50,0.0,6.792466,302.005341,solna
5,2025-11-18,0.10,0.0,10.308831,282.094727,solna
6,2025-11-19,0.90,0.4,12.245293,65.695465,solna


In [ ]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 352.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [ ]:
# Insert new data
air_quality_fg.insert(aq_today_df)

#For C-level
air_quality_fg_with_lagged.insert(aq__with_lagg_today_df)

2025-11-13 09:14:19,753 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1272015/fs/1258614/fg/1668524


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_1_offline_fg_materialization


%6|1763021670.323|FAIL|rdkafka#producer-9| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 50180ms in state UP, 1 identical error(s) suppressed)


Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1272015/jobs/named/air_quality_1_offline_fg_materialization/executions
2025-11-13 09:14:34,096 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1272015/fs/1258614/fg/1703336


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_including_lagged_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1272015/jobs/named/air_quality_including_lagged_1_offline_fg_materialization/executions


(Job('air_quality_including_lagged_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 747587
         }
       },
       "result": {
         "observed_value": 4.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-13T08:14:34.000095Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "suc

In [ ]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-13 09:14:48,274 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1272015/fs/1258614/fg/1668525


%6|1763021689.975|FAIL|rdkafka#consumer-10| [thrd:GroupCoordinator]: GroupCoordinator: 51.161.81.208:9093: Disconnected (after 50116ms in state UP, 1 identical error(s) suppressed)
%6|1763021689.975|FAIL|rdkafka#consumer-10| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 50117ms in state UP, 1 identical error(s) suppressed)
Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1272015/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-13 09:15:05,525 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-13 09:15:08,753 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED


%6|1763021721.443|FAIL|rdkafka#producer-9| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 50128ms in state UP, 1 identical error(s) suppressed)
%6|1763021740.464|FAIL|rdkafka#consumer-10| [thrd:GroupCoordinator]: GroupCoordinator: 51.161.81.208:9093: Disconnected (after 50000ms in state UP, 1 identical error(s) suppressed)
%6|1763021740.705|FAIL|rdkafka#consumer-10| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 50241ms in state UP, 1 identical error(s) suppressed)
%6|1763021791.053|FAIL|rdkafka#consumer-10| [thrd:GroupCoordinator]: GroupCoordinator: 51.161.81.208:9093: Disconnected (after 50131ms in state UP, 1 identical error(s) suppressed)


2025-11-13 09:16:41,927 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2025-11-13 09:16:48,333 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-13 09:16:48,497 INFO: Waiting for log aggregation to finish.


%6|1763021816.858|FAIL|rdkafka#producer-9| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 94377ms in state UP, 1 identical error(s) suppressed)


2025-11-13 09:16:57,150 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 739347
         }
       },
       "result": {
         "observed_value": 6.792466163635254,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-13T08:14:48.000273Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": 

%6|1763021841.177|FAIL|rdkafka#consumer-10| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 150415ms in state UP, 1 identical error(s) suppressed)
%6|1763021841.655|FAIL|rdkafka#consumer-10| [thrd:GroupCoordinator]: GroupCoordinator: 51.161.81.208:9093: Disconnected (after 50120ms in state UP, 1 identical error(s) suppressed)
%6|1763021867.983|FAIL|rdkafka#producer-9| [thrd:ssl://51.161.81.188:9093/bootstrap]: ssl://51.161.81.188:9093/1: Disconnected (after 50122ms in state UP, 1 identical error(s) suppressed)
%6|1763021892.329|FAIL|rdkafka#consumer-10| [thrd:GroupCoordinator]: GroupCoordinator: 51.161.81.208:9093: Disconnected (after 50210ms in state UP, 1 identical error(s) suppressed)
%6|1763021892.329|FAIL|rdkafka#consumer-10| [thrd:ssl://51.161.81.208:9093/bootstrap]: ssl://51.161.81.208:9093/2: Disconnected (after 50210ms in state UP, 1 identical error(s) suppressed)
%6|1763021892.400|FAIL|rdkafka#consumer-10| [thrd:ssl://51.161.80.189:9

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
